How does the Free Throw shooting percent develop over time?

 - Does it go down as the game approaches the ending due to higher pressure?
 - Does it go up thanks to players being warmer, or alternatively - better shooters take the ball?

I Also recommend Sebastian's script who explored some of things I intended to (mostly the "Hot Hand" phenomenon and other aspects of performance under pressure):

https://www.kaggle.com/sebastianmantey/d/sebastianmantey/nba-free-throws/eda-nba-free-throws

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

df = pd.read_csv("../input/free_throws.csv")

Let's create some new features - number of seconds and minutes from the beginning of the match:

In [ ]:
df['minute'] = df.time.apply(lambda x: int(x[:len(x)-3]))
df['sec'] = df.time.apply(lambda x: int(x[len(x)-2:]))
df['abs_min'] = 12 - df['minute']+12*(df.period -1)
df['abs_time'] = 60*(df.abs_min-1) + 60 - df['sec']

In [ ]:
def group_values(df,minute):
    made = len(df[(df.abs_min == minute) & (df.shot_made == 1)])
    total = len(df[df.abs_min == minute])
    return np.true_divide(made,total)

minutes = range(int(max(df.abs_min)))

per_min = []
for minu in minutes:
    per_min.append(group_values(df,minu))

Now Let's plot the shooting percentage over time:

In [ ]:
plt.plot(minutes,per_min)
plt.title('Scoring % over time - Always worse at the beginning')
plt.xlim([1,48])
plt.ylim([0.65,0.85])
plt.plot([12,12],[0,1], '--', linewidth = 1, color = 'r')
plt.plot([24,24],[0,1], '--', linewidth = 1, color = 'r')
plt.plot([36,36],[0,1], '--', linewidth = 1, color = 'r')
plt.plot([48,48],[0,1], '--', linewidth = 1, color = 'r')
plt.xlabel('Minute')
plt.ylabel('Free Throws %')

Interestingly, while the curve is relatively flat, 2 trends can be observed:

 1. At the beginning of every querter - the percentage drops
 2. At the end of a quarter, and by far most clearly seen at the end of the game, the shooting percentage goes up

It is possible that the number of shots also varies over time, and therefore the statistics can get less/more noisy, let's explore this:

In [ ]:
minutes_df = pd.DataFrame()
minutes_df['minutes'] = range(int(max(df.abs_min)))
minutes_df['shots'] = minutes_df.minutes.apply(lambda x: len(df[df.abs_min == x]))
minutes_df['players_num'] = minutes_df.minutes.apply(lambda x: len(np.unique(df.player[df.abs_min == x])))

In [ ]:
plt.plot(minutes_df['shots'])
plt.title('Number of Shots over time - mostly in the end of the querter')
plt.ylabel('# of Shots')
plt.xlabel('Minute')
plt.xlim([1,48])
plt.plot([12,12],[0,40000], '--', linewidth = 1, color = 'r')
plt.plot([24,24],[0,40000], '--', linewidth = 1, color = 'r')
plt.plot([36,36],[0,40000], '--', linewidth = 1, color = 'r')
plt.plot([48,48],[0,40000], '--', linewidth = 1, color = 'r')

We can see that every quarter the number of shots goes up as the quarter advances. again the last quarter is an anomaly - where the number of shots goes up significantly.

This makes sense taking into account that after a certain limit of fouls per quarter, the attacker goes to the line after every foul.

However - While we would expect the statistics of shots approaching the end of the quarter to be more consistent thanks to the larger number of samples, this does not explain the consistent difference between FT% at the beginning and the end of each quarter.

In [ ]:
players_df = pd.DataFrame()
players_df['name'] = np.unique(df.player)
players_df['pct'] = players_df.name.apply(lambda x: np.true_divide(len(df[(df.shot_made == 1) & (df.player ==x)]), len(df[df.player == x])))
df['player_pct'] = df.player.apply(lambda x: players_df.pct[players_df.name == x].values[0])
minutes_df['avg_pct'] = minutes_df.minutes.apply(lambda x: np.mean(df.player_pct[df.abs_min == x]))

Another possibility is that better FT shooters take the ball more often as the end of the quarter approaches and the offense is more likely to go to the line. let's explore which players shoot when:

In [ ]:
plt.plot(minutes_df.avg_pct)
plt.xlim([2,48])
plt.ylim([0.65,0.85])
plt.plot([12,12],[0,1], '--', linewidth = 1, color = 'r')
plt.plot([24,24],[0,1], '--', linewidth = 1, color = 'r')
plt.plot([36,36],[0,1], '--', linewidth = 1, color = 'r')
plt.plot([48,48],[0,1], '--', linewidth = 1, color = 'r')
plt.xlabel('Minute')
plt.ylabel('Average FT% of players shooting')
plt.title('In the money time - the better shooters go to the line')

We can see in the graph above that indeed better shooters go to the line in the end of the quarter and especially in the last minute of the game. 

We also see that the phenomenon of worse shooting percentage at the beginning of the game is dminished (though not sure if totally disappears). This implies that while the rising FT% at the end of the quarter probably results from a change in the shooters "demographics" (better shooters take the ball), the phenomenon of low FT% at the beginning is less player dependent, and therefore more time dependent (physical\psychological warm up).

This can be concluded if we check how the FT% of the player improves (or not) after the first 2-3 shots, on which I am working now :)